In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, concatenate, UpSampling2D, BatchNormalization

def conv_block(input_tensor, filters, kernel_size=3, padding='same', activation='relu'):
    x = Conv2D(filters, kernel_size, padding=padding)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(filters, kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    return x

def build_unet_plusplus(input_shape):
    inputs = Input(input_shape)

    # 第一层下采样
    conv1_1 = conv_block(inputs, 32)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1_1)

    # 第二层下采样
    conv2_1 = conv_block(pool1, 64)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2_1)

    # 第三层下采样
    conv3_1 = conv_block(pool2, 128)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3_1)

    # 第四层下采样
    conv4_1 = conv_block(pool3, 256)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4_1)

    # 中间层
    conv5_1 = conv_block(pool4, 512)

    # 第四层上采样
    up4_2 = UpSampling2D(size=(2, 2))(conv5_1)
    up4_2 = concatenate([up4_2, conv4_1], axis=3)
    conv4_2 = conv_block(up4_2, 256)

    # 第三层上采样
    up3_3 = UpSampling2D(size=(2, 2))(conv4_2)
    up3_3 = concatenate([up3_3, conv3_1], axis=3)
    conv3_3 = conv_block(up3_3, 128)

    # 第二层上采样
    up2_4 = UpSampling2D(size=(2, 2))(conv3_3)
    up2_4 = concatenate([up2_4, conv2_1], axis=3)
    conv2_4 = conv_block(up2_4, 64)

    # 第一层上采样
    up1_5 = UpSampling2D(size=(2, 2))(conv2_4)
    up1_5 = concatenate([up1_5, conv1_1], axis=3)
    conv1_5 = conv_block(up1_5, 32)

    # 输出层
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv1_5)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# 示例代码
model = build_unet_plusplus(input_shape=(256, 256, 3))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 32  896         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 256, 256, 32  128        ['conv2d_1[0][0]']               
 rmalization)                   )                                                             

                                                                                                  
 activation_8 (Activation)      (None, 16, 16, 512)  0           ['batch_normalization_9[0][0]']  
                                                                                                  
 conv2d_10 (Conv2D)             (None, 16, 16, 512)  2359808     ['activation_8[0][0]']           
                                                                                                  
 batch_normalization_10 (BatchN  (None, 16, 16, 512)  2048       ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_9 (Activation)      (None, 16, 16, 512)  0           ['batch_normalization_10[0][0]'] 
                                                                                                  
 up_sampli

                                                                                                  
 activation_16 (Activation)     (None, 256, 256, 32  0           ['batch_normalization_17[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_18 (Conv2D)             (None, 256, 256, 32  9248        ['activation_16[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_18 (BatchN  (None, 256, 256, 32  128        ['conv2d_18[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 activatio

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D, Concatenate

def conv_block(x, filters, kernel_size):
    x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(x)
    x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(x)
    return x

def upsample_block(x, skip, filters, kernel_size):
    x = UpSampling1D(size=2)(x)
    x = Concatenate()([x, skip])
    x = conv_block(x, filters, kernel_size)
    return x

def build_unet_plus_plus(input_shape):
    inputs = Input(shape=input_shape)
    # 编码器
    conv1 = conv_block(inputs, 32, 3)
    pool1 = MaxPooling1D(pool_size=2)(conv1)

    conv2 = conv_block(pool1, 64, 3)
    pool2 = MaxPooling1D(pool_size=2)(conv2)

    conv3 = conv_block(pool2, 128, 3)
    pool3 = MaxPooling1D(pool_size=2)(conv3)

    conv4 = conv_block(pool3, 256, 3)
    pool4 = MaxPooling1D(pool_size=2)(conv4)

    # 解码器
    up4 = upsample_block(conv4, conv3, 256, 3)
    up3 = upsample_block(up4, conv2, 128, 3)
    up2 = upsample_block(up3, conv1, 64, 3)
    up1 = upsample_block(up2, inputs, 32, 3)

    # 输出层
    outputs = Conv1D(filters=1, kernel_size=1, activation='sigmoid')(up1)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model
model = build_unet_plus_plus(input_shape=(256, 256, 3))
model.summary()

ValueError: Input 0 of layer "max_pooling1d" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 256, 256, 32)